# LangGraph MongoDB Integration - Agentic RAG

This notebook is a companion to the [Build AI Agents with LangGraph and MongoDB](https://www.mongodb.com/docs/atlas/ai-integrations/langgraph/build-agents) page. For the complete Python application and detailed explanations of the code, refer to the tutorial on the page.

This notebook takes you through how to use LangGraph to implement agentic RAG and agent memory (short and long-term) 
by using MongoDB as the vector database, LangChain to implement retrieval tools, and LangGraph to orchestrate the agent workflow.

<a target="_blank" href="https://colab.research.google.com/github/mongodb/docs-notebooks/blob/main/ai-integrations/langgraph.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Set up the environment

Check the requirements of the
[langchain-voyageai](https://pypi.org/project/langchain-voyageai/) package
to ensure you're using a compatible Python version.

Before you begin, load the `sample_mflix` dataset into your MongoDB cluster if you haven't already. To learn more, see [Sample Data](https://www.mongodb.com/docs/atlas/sample-data/).

In [ ]:
pip install --quiet --upgrade langgraph langgraph-checkpoint-mongodb langgraph-store-mongodb langchain langchain-mongodb langchain-voyageai langchain-openai pymongo

In [ ]:
import os

os.environ["VOYAGE_API_KEY"] = "<voyage-api-key>"
os.environ["OPENAI_API_KEY"] = "<openai-api-key>"
MONGODB_URI = "<connection-string>"

## Use MongoDB as a vector database

In [ ]:
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_voyageai import VoyageAIEmbeddings

embedding_model = VoyageAIEmbeddings(
    model = "voyage-3-large",
    output_dimension = 2048
)

# Instantiate the vector store
vector_store = MongoDBAtlasVectorSearch.from_connection_string(
   connection_string = MONGODB_URI,
   namespace = "sample_mflix.embedded_movies",
   embedding = embedding_model,
   text_key = "plot",
   embedding_key = "plot_embedding_voyage_3_large",
   relevance_score_fn = "dotProduct"
)

In [ ]:
# Use helper method to create the vector search index
vector_store.create_vector_search_index(
   dimensions = 2048, # The dimensions of the vector embeddings to be indexed 
   wait_until_complete = 60 # Number of seconds to wait for the index to build (can take around a minute)
)

In [ ]:
from langchain_mongodb.index import create_fulltext_search_index
from pymongo import MongoClient

# Connect to your cluster
client = MongoClient(MONGODB_URI)
collection = client["sample_mflix"]["embedded_movies"]

# Use helper method to create the search index
create_fulltext_search_index( 
   collection = collection,
   field = "title",
   index_name = "search_index",
   wait_until_complete = 60 # Number of seconds to wait for the index to build (can take around a minute)
)

## Define search tools


In [ ]:
from langchain.agents import tool

# Define a vector search tool
@tool
def plot_search(user_query: str) -> str:
    """
    Retrieve information using vector search to answer a user query.
    """
    
    retriever = vector_store.as_retriever(
       search_type = "similarity",
       search_kwargs = { "k": 5 } # Retrieve top 5 most similar documents
    )

    results = retriever.invoke(user_query)
   
    # Concatenate the results into a string
    context = "\n\n".join([f"{doc.metadata['title']}: {doc.page_content}" for doc in results])
    return context

# Test the tool
test_results = plot_search.invoke("What are some movies that take place in the ocean?")
print(test_results)

In [ ]:
from langchain_mongodb.retrievers.full_text_search import MongoDBAtlasFullTextSearchRetriever

# Define a full-text search tool
@tool
def title_search(user_query: str) -> str:
    """
    Retrieve movie plot content based on the provided title.
    """
    
    # Initialize the retriever
    retriever = MongoDBAtlasFullTextSearchRetriever(
       collection = collection,            # MongoDB Collection
       search_field = "title",             # Name of the field to search
       search_index_name = "search_index", # Name of the search index
       top_k = 1,                          # Number of top results to return       
    ) 
    results = retriever.invoke(user_query)
   
    for doc in results:
      if doc:
          return doc.metadata["fullplot"]
      else:
          return "Movie not found"
  
# Test the tool     
title_search.invoke("What is the plot of Titanic?")

## Define memory tools (long-term memory)

In [ ]:
from langchain.agents import tool
from langgraph.store.mongodb import MongoDBStore, create_vector_index_config

# Vector search index configuration for memory collection
index_config = create_vector_index_config(
    embed=embedding_model,
    dims=2048,
    relevance_score_fn="dotProduct",
    fields=["content"]
)

# Define a tool to save important interactions to a MongoDB collection
@tool
def save_memory(content: str) -> str:
    """Save important information to memory."""
    with MongoDBStore.from_conn_string(
        conn_string=MONGODB_URI,
        db_name="sample_mflix",
        collection_name="memories",
        index_config=index_config,
        auto_index_timeout=60 # Wait a minute for vector index creation
    ) as store:
        store.put(
            namespace=("user", "memories"),
            key=f"memory_{hash(content)}",
            value={"content": content}
        )
    return f"Memory saved: {content}"

In [ ]:
# Define a tool to retrieve important interactions from the collection using vector search
@tool
def retrieve_memories(query: str) -> str:
    """Retrieve relevant memories based on a query."""
    with MongoDBStore.from_conn_string(
        conn_string=MONGODB_URI,
        db_name="sample_mflix",
        collection_name="memories",
        index_config=index_config
    ) as store:
        results = store.search(("user", "memories"), query=query, limit=3)

    if results:
        memories = [result.value["content"] for result in results]
        return f"Retrieved memories:\n" + "\n".join(memories)
    else:
        return "No relevant memories found."

In [ ]:
# Test the tools
save_memory.invoke("I don't like sad movies.")
retrieve_memories.invoke("What do you know about me?")

## Prepare the LLM

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# Initialize the LLM
llm = ChatOpenAI()

# Create a chat prompt template for the agent, which includes a system prompt and a placeholder for `messages`
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "You are a helpful AI chatbot."
            " You are provided with tools to answer questions about movies."
            " Think step-by-step and use these tools to get the information required to answer the user query."
            " Do not re-run tools unless absolutely necessary."
            " If you are not able to get enough information using the tools, reply with I DON'T KNOW."
            " You have access to the following tools: {tool_names}."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
 
tools = [
    plot_search,
    title_search,
    save_memory,
    retrieve_memories
]

# Provide the tool names to the prompt
prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))

# Prepare the LLM by making the tools and prompt available to the model
bind_tools = llm.bind_tools(tools)
llm_with_tools = prompt | bind_tools

In [ ]:
# Check the 'name' of the tool that the LLM is calling.
# Here, we expect the LLM to use the 'plot_search' tool.
llm_with_tools.invoke(["What are some movies that take place in the ocean?"]).tool_calls

In [ ]:
# Check the 'name' of the tool that the LLM is calling.
# Here, we expect the LLM to use the 'title_search' tool.
llm_with_tools.invoke(["What's the plot of Titanic?"]).tool_calls

In [ ]:
# Check the 'name' of the tool that the LLM is calling.
# Here, we expect the LLM to use the 'save_memory' tool.
llm_with_tools.invoke(["I prefer funny movies."]).tool_calls

In [ ]:
# Check the 'name' of the tool that the LLM is calling.
# Here, we expect the LLM to use the 'retrieve_memories' tool.
llm_with_tools.invoke(["What do you know about me?"]).tool_calls

## Build the graph

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages

# Define the graph state
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# Instantiate the graph
graph = StateGraph(GraphState)


In [ ]:
from typing import Dict, List

# Define the agent node
def agent(state: GraphState) -> Dict[str, List]:
    # Get the messages from the graph `state`
    messages = state["messages"]
    # Invoke `llm_with_tools` with `messages`
    result = llm_with_tools.invoke(messages)
    # Write `result` to the `messages` attribute of the graph state
    return {"messages": [result]}

# Add nodes using the `add_node` function
# The `agent` node should run the `agent` function
graph.add_node("agent", agent)

In [ ]:
from langchain_core.messages import ToolMessage

# Create a map of tool name to tool call
tools_by_name = {tool.name: tool for tool in tools}

# Define tools node
def tools_node(state: GraphState) -> Dict[str, List]:
    result = []
    # Get the list of tool calls from messages
    tool_calls = state["messages"][-1].tool_calls
    # Iterate through `tool_calls`
    for tool_call in tool_calls:
        # Get the tool from `tools_by_name` using the `name` attribute of the `tool_call`
        tool = tools_by_name[tool_call["name"]]
        # Invoke the `tool` using the `args` attribute of the `tool_call`
        observation = tool.invoke(tool_call["args"])
        # Append the result of executing the tool to the `result` list as a ToolMessage
        # The `content`` of the message is `observation`
        # The `tool_call_id` can be obtained from the `tool_call`
        result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
    # Write `result` to the `messages` attribute of the graph state
    return {"messages": result}

# The `tools` node should run the `tools_node` function
graph.add_node("tools", tools_node)

In [ ]:
from langgraph.graph import END

# Add an edge from the START node to the `agent` node
graph.add_edge(START, "agent")

# Add an edge from the `tools` node to the `agent` node
graph.add_edge("tools", "agent")

# Define a conditional edge
def route_tools(state: GraphState):
    """
    Uses a conditional_edge to route to the tools node if the last message
    has tool calls. Otherwise, route to the end.
    """
    # Get messages from graph state
    messages = state.get("messages", [])
    if len(messages) > 0:
        # Get the last AI message from messages
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    
    # Check if the last message has tool calls
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return END

# Add a conditional edge from the `agent` node to the `tools` node
graph.add_conditional_edges(
    "agent",
    route_tools,
    {"tools": "tools", END: END},
)

In [ ]:
from IPython.display import Image, display

# Compile the graph
app = graph.compile()

# Optionally, visualize the graph
try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    pass

## Execute the graph

In [ ]:
# Stream outputs from the graph as they pass through its nodes
def execute_graph(user_input: str) -> None:
    # Add user input to the messages attribute of the graph state
    # The role of the message should be "user" and content should be `user_input`
    input = {"messages": [("user", user_input)]}

    # Pass input to the graph and stream the outputs
    for output in app.stream(input):
        for key, value in output.items():
            print(f"Node {key}:")
            print(value)
            
    print("\n---FINAL ANSWER---")
    print(value["messages"][-1].content)

In [ ]:
# Test the graph execution to view end-to-end flow
execute_graph("What are some movies that take place in the ocean?")

In [ ]:
execute_graph("What is the plot of Titanic?")

In [ ]:
# Using long-term memory
execute_graph("Recommend me some movies based on what you know about me.")

## Enable persistence (short-term memory)

In [ ]:
from langgraph.checkpoint.mongodb import MongoDBSaver

# Initialize a MongoDB checkpointer
checkpointer = MongoDBSaver(client)

# Instantiate the graph with the checkpointer
app = graph.compile(checkpointer=checkpointer)

In [ ]:
# Update the `execute_graph` function to include the `thread_id` argument
def execute_graph(thread_id: str, user_input: str) -> None:
    config = {"configurable": {"thread_id": thread_id}}
    input = {
        "messages": [
            (
                "user",
                user_input,
            )
        ]
    }
    for output in app.stream(input, config):
        for key, value in output.items():
            print(f"Node {key}:")
            print(value)
            
    print("\n---FINAL ANSWER---")
    print(value["messages"][-1].content)

In [ ]:
# Test graph execution with thread ID
execute_graph("1", "What's the plot of Titanic?")

In [ ]:
# Follow-up question to ensure message history works
execute_graph("1", "What movies are similar to the one I just asked about?")

In [ ]:
# Ask question in new session to show that long-term memory works across sessions
execute_graph("2", "Recommend me movies based on what you know about me.")